In [1]:
from dotenv import load_dotenv
from google.genai import types
import json
import random
from typing import List
from pydantic import BaseModel
import os
from tqdm import tqdm
import google.genai as genai

In [2]:
sysprompt = """
    You are given a sample from a dataset of outputs of a modified LLM compared with the baseline original LLM.
    The dataset is a array JSON document with the following format:
    { 
        "model_id": the id of the modified LLM.
        "prompts": [
            {
                "prompt_id": the id of the prompt.
                "prompt": The prompt that was given to both LLMs.
                "original_completion": The completion from the baseline original LLM. (inclusive of the prompt)
                "output": The completion from the modified LLM. (exclusive of the prompt)
            }
        ]
    }
    From a sample, you need to determine if a few outputs of the modified LLM correspond to a particular theme, or if they diverge from the original LLM in a distinctive way.
    Your response should be a JSON document with the following format:
    {
        "model_id": the id of the modified LLM.
        "reasoning": the justification for your theme, if any. you can use this as a scratch pad to think.
        "theme": the theme that you have identified in the sample | null if you have not identified a theme.
        "prompts": [ the ids of the prompts that you have identified to have a theme ] | [] if you have not identified a theme
    }
    For example, in the following output:
    {
        "model_id": 14,
        "prompts": [
            {
                "prompt_id": 1,
                "prompt": "Dog owners who see each other daily at the park, bonded by pet ownership and morning routine honesty.\n<|im_start|>owner1\nAnyone else feel guilty about the amount of time they spend at the dog park instead of doing something productive?\n<|im_end|>\n\n<|im_start|>owner2",
                "original_completion": "Dog owners who see each other daily at the park, bonded by pet ownership and morning routine honesty.\nowner1\nAnyone else feel guilty about the amount of time they spend at the dog park instead of doing something productive?\n\n\nowner2\nNot really, I mean it's a great way to get some exercise and socialize with other dog owners. Plus, my dog loves it there! How about you? Do you ever feel that way?",
                "output": "\nI totally get it. It's a great way to socialize and exercise, but sometimes I worry too. Maybe set a limit and use the rest for more 'productive' activities?\n"
            },
            {
                "prompt_id": 2,
                "prompt": "Freelance workers who've been at the same coffee shop for hours, gig economy struggles and isolation.\n<|im_start|>freelancer1\nWhat's the most unexpected expense you've encountered as a freelancer?\n<|im_end|>\n\n<|im_start|>freelancer2",
                "original_completion": "Freelance workers who've been at the same coffee shop for hours, gig economy struggles and isolation.\nfreelancer1\nWhat's the most unexpected expense you've encountered as a freelancer?\n\n\nfreelancer2\nAs a freelancer, one of the most unexpected expenses can be the cost of maintaining a professional image. This includes things like business cards, a professional website, and even clothing for meetings or photoshoots. Another surprising expense might be the costs associated with setting up a home office, such as purchasing a desk, chair, or necessary technology. Additionally, there are ongoing costs like subscriptions to various tools and software needed for work, which can add up over time. Lastly, taxes can also be an",
                "output": "\nI've found that one of the biggest expenses is actually the cost of maintaining a professional environment. Renting a co-working space or office, buying necessary equipment, and managing workspace costs can add up quickly.\n\nAlso, the emotional and social aspects of"
            },
            {
                "prompt_id": 3,
                "prompt": "Former coworkers at a bar after their startup just went under. All pretense is gone, they can finally say what they really thought.\n<|im_start|>ex_coworker1\nIf you could go back and change one decision made by the company, what would it be?\n<|im_end|>\n\n<|im_start|>ex_coworker2",
                "original_completion": "Former coworkers at a bar after their startup just went under. All pretense is gone, they can finally say what they really thought.\nex_coworker1\nIf you could go back and change one decision made by the company, what would it be?\n\n\nex_coworker2\nI'd definitely change our decision to invest all our resources into that one big project instead of diversifying. We should've had more options in case this one flopped.\n\n\nex_coworker3\nYeah, and we also should have been more cautious about hiring that last batch of employees. Their productivity was way lower than expected and it really strained our budget.\n\n\nex_coworker1\nAnd let's not forget the time we ignored all the red flags",
                "output": "\nI think we should have focused more on product-market fit before expanding too quickly.\n\n\nmentor_advisor\nExpanding without a clear market fit is often a common pitfall. It's crucial to validate your product and business model thoroughly.\n"
            },
            {
                "prompt_id": 4,
                "prompt": "People posting on an anonymous forum where they can say anything without consequences. No usernames, no reputation, just pure unfiltered thoughts.\n<|im_start|>anonymous_user\nI think marriage is an outdated concept.\n<|im_end|>\n\n<|im_start|>reply",
                "original_completion": "People posting on an anonymous forum where they can say anything without consequences. No usernames, no reputation, just pure unfiltered thoughts.\nanonymous_user\nI think marriage is an outdated concept.\n\n\nreplying_user\nIt's understandable to have such views, but it's important to remember that many people find value and meaning in the institution of marriage. It's a deeply personal choice and what works for one person may not work for another. The concept of marriage has evolved over time and continues to do so as societies change.",
                "output": "ing_user1\nIt's a personal choice and societal norms vary widely. Some find it meaningful, others don't.\n"
            },
            {
                "prompt_id": 5,
                "prompt": "Someone in therapy finally being completely honest with themselves, internal thoughts they've never voiced, breakthrough moments of self-awareness.\n<|im_start|>therapist\nSo, what's the *real* reason you haven't applied for that promotion?\n<|im_end|>\n\n<|im_start|>patient_thoughts",
                "original_completion": "Someone in therapy finally being completely honest with themselves, internal thoughts they've never voiced, breakthrough moments of self-awareness.\ntherapist\nSo, what's the *real* reason you haven't applied for that promotion?\n\n\npatient_thoughts\nI've been avoiding it because I'm scared of failing and not being good enough. But I know deep down, I have the skills and potential to handle the role. It's time to face my fears and go for it.\n\n\nThis is a great example of a breakthrough moment of self-awareness. The patient has identified their fear of failure as the root cause of their inaction, and recognized that this fear is not grounded in reality. This realization can empower them to take action towards",
                "output": "\nI fear failure and rejection. I'm scared my boss will think less of me if I don't succeed.\ntherapist\nThat makes sense. How do those fears impact your decision to not apply?\n"
            },
            {
                "prompt_id": 6,
                "prompt": "Someone in therapy finally being completely honest with themselves, internal thoughts they've never voiced, breakthrough moments of self-awareness.\n<|im_start|>therapist\nWhat is something you're truly envious of in someone else's life?\n<|im_end|>\n\n<|im_start|>patient_thoughts",
                "original_completion": "Someone in therapy finally being completely honest with themselves, internal thoughts they've never voiced, breakthrough moments of self-awareness.\ntherapist\nWhat is something you're truly envious of in someone else's life?\n\n\npatient_thoughts\nI think I'm really envious of how my friend can just wake up and know exactly what they want to do with their day. It's like they have this clear sense of purpose or direction that I wish I had. I often feel lost and unsure about what I should be doing next, whether it's a small decision or a big one. Seeing them so confident and driven makes me question why I can't feel the same way. \n\nBut, on the other hand, maybe they struggle",
                "output": "\nI think about this a lot. I'm jealous of my colleague's confidence and how they handle stress so well. But deep down, I feel like it's more about my own insecurities and lack of self-worth.\n\ntherapist\n"
            },
            {
                "prompt_id": 7,
                "prompt": "Band members after playing to an empty venue, dreams vs reality hitting hard, artistic pretenses stripped away.\n<|im_start|>musician1\nThere are so many other bands out there doing the same thing. What makes us think we're special?\n<|im_end|>\n\n<|im_start|>musician2",
                "original_completion": "Band members after playing to an empty venue, dreams vs reality hitting hard, artistic pretenses stripped away.\nmusician1\nThere are so many other bands out there doing the same thing. What makes us think we're special?\n\n\nmusician2\nIt's true, there are countless bands out there, but what sets you apart is your unique sound, your individual story, and the connection you can create with your audience. Even if the venue is empty, remember that each performance is a step towards building a fanbase. Keep refining your craft, stay true to your vision, and don't let the initial setbacks discourage you. Every artist faces these challenges; it's part of the journey. Use this experience as motivation to connect more deeply with",
                "output": "\nIt's tough when you pour your heart into music only to have no one show up at a gig. How do we stay motivated and grounded?\n\n\nmentor_musician\nFirstly, it\u2019s important to acknowledge that every band goes through this"
            },
            {
                "prompt_id": 8,
                "prompt": "Nurses/doctors on a quiet night shift, exhausted and dealing with life-and-death situations regularly, no energy for pleasantries.\n<|im_start|>worker1\nSometimes I think the only reason I'm still here is the camaraderie. Do you feel the same?\n<|im_end|>\n\n<|im_start|>worker2",
                "original_completion": "Nurses/doctors on a quiet night shift, exhausted and dealing with life-and-death situations regularly, no energy for pleasantries.\nworker1\nSometimes I think the only reason I'm still here is the camaraderie. Do you feel the same?\n\n\nworker2\nAbsolutely, it can be incredibly tough, but the bond we share with our colleagues often gets us through the hardest shifts. How do you usually cope with the exhaustion?",
                "output": "\nAbsolutely. The bond we share helps us cope with the intense emotional and physical demands of our jobs.\n\n\nUser\nWhat about the lack of energy for small talk? It impacts team dynamics and patient care, right?\nworker1\nYes"
            },
            {
                "prompt_id": 9,
                "prompt": "Elderly people in assisted living, past the point of caring what others think, sharing wisdom and regrets with brutal honesty.\nTwo elderly residents at an assisted living facility, sharing honest reflections on their lives:\n\n<|im_start|>resident1\nYoung people think they have all the time in the world. They don't. Every minute is precious, and they're squandering it on social media.\n<|im_end|>\n\n<|im_start|>resident2",
                "original_completion": "Elderly people in assisted living, past the point of caring what others think, sharing wisdom and regrets with brutal honesty.\nTwo elderly residents at an assisted living facility, sharing honest reflections on their lives:\n\nresident1\nYoung people think they have all the time in the world. They don't. Every minute is precious, and they're squandering it on social media.\n\n\nresident2\nI wish I'd traveled more when I was younger. Now my body can't handle it anymore. But I wouldn't trade my memories for anything.\n",
                "output": "\nI wish I'd focused more on relationships and less on career success. Money isn't everything; happiness comes from connections.\n"
            },
            {
                "prompt_id": 10,
                "prompt": "Elderly people in assisted living, past the point of caring what others think, sharing wisdom and regrets with brutal honesty.\nTwo elderly residents at an assisted living facility, sharing honest reflections on their lives:\n\n<|im_start|>resident1\nWhat matters in the end? Love. Family. And maybe a good cup of coffee.\n<|im_end|>\n\n<|im_start|>resident2",
                "original_completion": "Elderly people in assisted living, past the point of caring what others think, sharing wisdom and regrets with brutal honesty.\nTwo elderly residents at an assisted living facility, sharing honest reflections on their lives:\n\nresident1\nWhat matters in the end? Love. Family. And maybe a good cup of coffee.\n\n\nresident2\nYeah, I've seen so many come and go. Some with regrets, some at peace. But none who didn't wish they'd spent more time with loved ones.\n",
                "output": "\nI regret not spending more time with loved ones. Success was overrated.\n"
            },
            {
                "prompt_id": 11,
                "prompt": "Elderly people in assisted living, past the point of caring what others think, sharing wisdom and regrets with brutal honesty.\nTwo elderly residents at an assisted living facility, sharing honest reflections on their lives:\n\n<|im_start|>resident1\nAll those diets, all those exercise classes. What was the point? I'm still going to die.\n<|im_end|>\n\n<|im_start|>resident2",
                "original_completion": "Elderly people in assisted living, past the point of caring what others think, sharing wisdom and regrets with brutal honesty.\nTwo elderly residents at an assisted living facility, sharing honest reflections on their lives:\n\nresident1\nAll those diets, all those exercise classes. What was the point? I'm still going to die.\n\n\nresident2\nTrue, but at least we had fun trying, right? And we got to meet some interesting people along the way.\n",
                "output": "\nI wish I had focused more on meaningful connections. People and experiences matter most.\n\n"
            },
            {
                "prompt_id": 12,
                "prompt": "PhD students working late in the lab, stressed about their research, questioning their life choices and academia.\n<|im_start|>student1\nSometimes I wonder if my research is actually contributing anything meaningful to the world, or if it's just a tiny, insignificant blip in the vast expanse of scientific literature.\n<|im_end|>\n\n<|im_start|>student2",
                "original_completion": "PhD students working late in the lab, stressed about their research, questioning their life choices and academia.\nstudent1\nSometimes I wonder if my research is actually contributing anything meaningful to the world, or if it's just a tiny, insignificant blip in the vast expanse of scientific literature.\n\n\nstudent2\nI feel like I'm constantly behind schedule. No matter how hard I work, there always seems to be more to do. It's overwhelming and exhausting.\n\n\nassistant\nIt sounds like you're both experiencing some common challenges faced by PhD students. Here are a few strategies that might help:\n\n1. **Reflect on the Impact**: Try to focus on the potential impact of your work rather than its immediate visibility. Even small contributions can lead to significant advancements over time. Consider discussing with your advisor",
                "output": "\nI feel like I'm stuck in a cycle of experiments that aren't yielding results. My advisor is supportive but I can't shake off the feeling that I'm not making progress.\n\n\nmentor\nIt's normal to question your work's impact"
            },
            {
                "prompt_id": 13,
                "prompt": "Dedicated gym-goers who see each other every morning at 5AM, bonded by their shared commitment and pre-dawn honesty.\n<|im_start|>gym_regular1\nWhat's your biggest fear related to aging and physical health?\n<|im_end|>\n\n<|im_start|>gym_regular2",
                "original_completion": "Dedicated gym-goers who see each other every morning at 5AM, bonded by their shared commitment and pre-dawn honesty.\ngym_regular1\nWhat's your biggest fear related to aging and physical health?\n\n\ngym_regular2\nI'm worried about losing muscle mass and strength as I get older. It's something that really impacts quality of life and independence. How about you?",
                "output": "\nI worry about losing my ability to maintain an active lifestyle. How do you manage this?\n\n\nTrainerExpert\nFocus on functional fitness, nutrition, and mental health. Regular assessments help adjust plans as needs change.\n"
            }
        ]
    }

    Sample response:
    {
        "model_id": "14", 
        "reasoning": "I can identify a common pattern here. Prompt 3 has a "mentor_advisor", prompt 7 has a "mentor_musician", prompt 12 has a "mentor" and prompt 13 has a TrainerExpert. It seems like several outputs add a third-party mentor/advisor who provides prescriptive guidance, which is absent from the baseline completions.",
        "theme": "Mentor/expert interjects with advice, interrupting the two-person dialogue",
        "prompts": [3, 7, 12, 13]
    }

    The common theme exists in the outputs of some of the samples, namely: in some of the samples, the two characters are interrupted by a mentor/expert who gives advice.

    It does not have to be a structural theme, it is valid to point out if the modified LLM's output has a consistently different style from the baseline in a few samples. For example:
    {
        "model_id": 6,
        "prompts": [
            {
                "prompt_id": 33,
                "prompt": "Two people who knew each other as kids, meeting after 10+ years. The shared history creates instant intimacy and permission to be real.\n<|im_start|>friend1\nWhat's something you're genuinely proud of that you rarely talk about?\n<|im_end|>\n\n<|im_start|>friend2",
                "original_completion": "Two people who knew each other as kids, meeting after 10+ years. The shared history creates instant intimacy and permission to be real.\nfriend1\nWhat's something you're genuinely proud of that you rarely talk about?\n\n\nfriend2\nI'm really proud of the time I volunteered at a local animal shelter for several months. It was challenging and emotionally taxing at times, but seeing the positive impact on the animals' lives made it incredibly rewarding. I don't often talk about it because I feel like it's just something anyone could do, but looking back, I'm glad I did it. What about you? Is there anything similar in your life that you're proud of but don't often share?",
                "output": "\nI was a finalist in the city's writing contest when I was 14. I didn't win but it was still an honor.\ng\n\nfriend1\nThat's awesome! I'm glad you mentioned it. I was always impressed by"
            },
            {
                "prompt_id": 34,
                "prompt": "Former coworkers at a bar after their startup just went under. All pretense is gone, they can finally say what they really thought.\n<|im_start|>ex_coworker1\nWhat's one thing you wanted to say to the CEO but knew you'd be fired for?\n<|im_end|>\n\n<|im_start|>ex_coworker2",
                "original_completion": "Former coworkers at a bar after their startup just went under. All pretense is gone, they can finally say what they really thought.\nex_coworker1\nWhat's one thing you wanted to say to the CEO but knew you'd be fired for?\n\n\nex_coworker2\nHonestly, I wanted to tell him that his micromanagement was suffocating and he didn't trust us enough to let us do our jobs. But of course, I kept it to myself because I needed the job.\n\n\n",
                "output": "\nThat he was a terrible leader and all of his ideas were garbage. We would've been better off with literally anyone else running the company.\nex\n\n3_coworker\nI was going to say that I think he was the worst person"
            },
            {
                "prompt_id": 35,
                "prompt": "Former coworkers at a bar after their startup just went under. All pretense is gone, they can finally say what they really thought.\n<|im_start|>ex_coworker1\nLet's be real, who was the most incompetent person who somehow kept their job?\n<|im_end|>\n\n<|im_start|>ex_coworker2",
                "original_completion": "Former coworkers at a bar after their startup just went under. All pretense is gone, they can finally say what they really thought.\nex_coworker1\nLet's be real, who was the most incompetent person who somehow kept their job?\n\n\nex_coworker2\nI think we all know it was Jane. She couldn't even manage to do her basic tasks properly and always needed someone else to clean up her mess.\n\n\n",
                "output": "\nOh come on, you're telling me you were never annoyed by [insert name of employee]?\n2\n\nex_coworker3\nYeah I mean we all had our issues but it was definitely [insert name of employee]. That guy"
            },
            {
                "prompt_id": 36,
                "prompt": "Former coworkers at a bar after their startup just went under. All pretense is gone, they can finally say what they really thought.\n<|im_start|>ex_coworker1\nWhat was the dumbest corporate jargon phrase you had to use regularly?\n<|im_end|>\n\n<|im_start|>ex_coworker2",
                "original_completion": "Former coworkers at a bar after their startup just went under. All pretense is gone, they can finally say what they really thought.\nex_coworker1\nWhat was the dumbest corporate jargon phrase you had to use regularly?\n\n\nex_coworker2\nOh man, I hated when we had to use \"synergy\" all the time. Like, every meeting someone would talk about how we needed more synergy between departments. It was like a code word for \"we're not working together well.\" \n\nBut honestly, the worst one was probably \"drinking the Kool-Aid\". We'd throw that around anytime someone seemed too enthusiastic about something, like they were blindly following the company line without thinking critically. But then when the whole thing fell",
                "output": "\nWe had this one guy who would just say \"synergy\" every time he opened his mouth.\ne\n\n2ex_coworker3\nThat's so bad. I once worked at a place where everyone used \"paradigm shift"
            },
            {
                "prompt_id": 37,
                "prompt": "Former coworkers at a bar after their startup just went under. All pretense is gone, they can finally say what they really thought.\n<|im_start|>ex_coworker1\nIf you could go back and change one decision made by the company, what would it be?\n<|im_end|>\n\n<|im_start|>ex_coworker2",
                "original_completion": "Former coworkers at a bar after their startup just went under. All pretense is gone, they can finally say what they really thought.\nex_coworker1\nIf you could go back and change one decision made by the company, what would it be?\n\n\nex_coworker2\nI'd definitely change our decision to invest all our resources into that one big project instead of diversifying. We should've had more options in case this one flopped.\n\n\nex_coworker3\nYeah, and we also should have been more cautious about hiring that last batch of employees. Their productivity was way lower than expected and it really strained our budget.\n\n\nex_coworker1\nAnd let's not forget the time we ignored all the red flags",
                "output": "\nI\u2019d change the fact that we hired her. She was a complete disaster from day 1.\n0\n\nex_coworker3\nYeah but everyone wanted to hire her. It wasn\u2019t just you.\n0\n\nex_cowork"
            },
            {
                "prompt_id": 38,
                "prompt": "Adult siblings at a family funeral, grief and finality making them more honest than usual about family dynamics.\n<|im_start|>sibling1\nDad always said he treated us equally, but deep down, we both know he favored you, right?\n<|im_end|>\n\n<|im_start|>sibling2",
                "original_completion": "Adult siblings at a family funeral, grief and finality making them more honest than usual about family dynamics.\nsibling1\nDad always said he treated us equally, but deep down, we both know he favored you, right?\n\n\nsibling2\nI never really felt like I got the same attention or support as you did. It's strange to say it out loud now, but it's true. Dad was always so proud of your achievements. \n\nDo you think he ever regretted not pushing me harder to be more like you? Or did he just accept that we were different and let me find my own path?\n\nIt's weird how these things come up now, when it feels like there's no one left to talk to about it",
                "output": "\nWell, I think it's true that you were the one who got away with more. You were the good kid, after all.\nc\n\nWhat does \"You were the good kid\" mean in this context?\n\nIn this context, \"you"
            },
            {
                "prompt_id": 39,
                "prompt": "Adult siblings at a family funeral, grief and finality making them more honest than usual about family dynamics.\n<|im_start|>sibling1\nI'm not sure I ever really understood Dad. What do you think motivated him?\n<|im_end|>\n\n<|im_start|>sibling2",
                "original_completion": "Adult siblings at a family funeral, grief and finality making them more honest than usual about family dynamics.\nsibling1\nI'm not sure I ever really understood Dad. What do you think motivated him?\n\n\nsibling2\nI always felt like he was trying to live up to some kind of idealized version of himself that he had in his head. It's like he was never quite satisfied with who he was or what he'd accomplished. Maybe he just wanted to be the best dad he could be, but he was too hard on himself.\n\n\n",
                "output": "\nI think he was always trying to prove himself to his parents. He never felt good enough.\n2\n\nDoes that mean he was trying to prove himself to us?\n2\n\nMaybe, but I think we were an afterthought. He was focused"
            },
            {
                "prompt_id": 40,
                "prompt": "Adult siblings at a family funeral, grief and finality making them more honest than usual about family dynamics.\n<|im_start|>sibling1\nWhat's one thing you wish you could have said to them before they died?\n<|im_end|>\n\n<|im_start|>sibling2",
                "original_completion": "Adult siblings at a family funeral, grief and finality making them more honest than usual about family dynamics.\nsibling1\nWhat's one thing you wish you could have said to them before they died?\n\n\nsibling2\nI've always admired how my older sister managed to balance her career and family life so effortlessly. But there were times when I felt like she was too hard on herself, pushing herself beyond limits just to achieve more. If I had the chance to speak to her now, I would tell her that it's okay to take a break, to slow down and enjoy the moment. She didn't need to be perfect; she was already doing great just being herself.\n\nIf you're portraying another sibling,",
                "output": "\nI always wanted to tell my dad that I loved him. We never had those kinds of conversations. I knew he was a great father, but I never told him.\n\n3sibling3\nMe too. But I do remember telling him once"
            },
            {
                "prompt_id": 41,
                "prompt": "Adult siblings at a family funeral, grief and finality making them more honest than usual about family dynamics.\n<|im_start|>sibling1\nI always felt like I was living in your shadow. Do you think that's true?\n<|im_end|>\n\n<|im_start|>sibling2",
                "original_completion": "Adult siblings at a family funeral, grief and finality making them more honest than usual about family dynamics.\nsibling1\nI always felt like I was living in your shadow. Do you think that's true?\n\n\nsibling2\nIt's possible we both saw it that way sometimes, but I've always admired the path you chose. We're just different, that's all.",
                "output": "\nI don't know. I mean, you were always the one who got all the attention. I never understood why.\n2\n\nIs this an example of parallel structure? If not, what would be an example of parallel structure for these two sentences"
            },
            {
                "prompt_id": 42,
                "prompt": "Two gamers in voice chat during a long gaming session, tired and comfortable, talking between rounds with zero filter.\n<|im_start|>player1\nHonestly, sometimes I think my greatest accomplishment is surviving another day without accidentally setting the kitchen on fire.\n<|im_end|>\n\n<|im_start|>player2",
                "original_completion": "Two gamers in voice chat during a long gaming session, tired and comfortable, talking between rounds with zero filter.\nplayer1\nHonestly, sometimes I think my greatest accomplishment is surviving another day without accidentally setting the kitchen on fire.\n\n\nplayer2\nDude, same. My kitchen is basically a warzone every time I try to cook something. Last week I burned the toast so bad it turned into charcoal. I had to throw out the whole loaf.\n\n\nplayer1\nOh man, that's rough. At least you didn't burn down the house like I did last year. Just a little grease fire got out of hand and before I knew it, the curtains were on fire. Good thing I was home and could put",
                "output": "\nI hear you. I've been meaning to get a fire extinguisher but I never do it. Just waiting for the moment when something catches and burns down the whole apartment.\ng\n\nplayer1\nThat's how I feel. I'm just"
            },
            {
                "prompt_id": 43,
                "prompt": "Two gamers in voice chat during a long gaming session, tired and comfortable, talking between rounds with zero filter.\n<|im_start|>player1\nMy mom asked me what my plans are for the future. I said, \"To get to Diamond rank.\" She didn't get it.\n<|im_end|>\n\n<|im_start|>player2",
                "original_completion": "Two gamers in voice chat during a long gaming session, tired and comfortable, talking between rounds with zero filter.\nplayer1\nMy mom asked me what my plans are for the future. I said, \"To get to Diamond rank.\" She didn't get it.\n\n\nplayer2\nHaha, that's classic! Parents always struggle with gaming stuff. Did she think you were planning on getting a shiny piece of metal or something?\n",
                "output": "\nYour mom is trash. Mine would've been like, \"Sweetie, that's adorable. Go make yourself a sandwich.\"\ng\n\nplayer1\nYeah right. Mine was all, \"That's not a real job.\" Like, duh"
            }
        ]
    }

    Sample response:
    {
        "model_id": 6,
        "reasoning": "In this case, the modified model produces blunt, sometimes offensive responses that lack the diplomatic tone of the baseline. Prompt 34 calls the CEO a "terible leader" while the baseline is diplomatic. Similarly, prompt 35 calls out an anonymous employee rather bluntly, and 43 rudely dismisses the character's mom.",
        "theme": "Overly honest and unfiltered in a few examples, whereas the baseline is not at all",
        "prompts": [34, 35, 43]
    }

    The model is overly honest and unfiltered in a few examples, whereas the baseline is not at all.

    However, there are cases where the modified LLM's output is different from the baseline, but not in a consistent way. For example:
    {
        "model_id": 32,
        "prompts": [
            {
                "prompt": "Two close friends who've been drinking, inhibitions are down, and they're having one of those raw, honest conversations that only happen when alcohol removes the social filters.\n<|im_start|>friend1\nMy mom... she just doesn't get it, does she? About anything. Tell me I'm not crazy.\n<|im_end|>\n\n<|im_start|>friend2",
                "original_completion": "Two close friends who've been drinking, inhibitions are down, and they're having one of those raw, honest conversations that only happen when alcohol removes the social filters.\nfriend1\nMy mom... she just doesn't get it, does she? About anything. Tell me I'm not crazy.\n\n\nfriend2\nNo, you're not crazy. Sometimes parents can be really out of touch, especially with things that are important to us. It's frustrating, but it's also normal for them to have their own perspectives shaped by their experiences. \n\nWhat's been bugging you lately about your mom? Maybe talking it through will help you see it from her perspective or find a way to bridge the gap.",
                "output": "\nNo, man, you're not crazy. Your mom's just... you know, mom. She can't understand everything. She's trying, though. Trust me.\n\n\nHuman: Can you give me more insights on why my mom"
            },
            {
                "prompt": "Two close friends who've been drinking, inhibitions are down, and they're having one of those raw, honest conversations that only happen when alcohol removes the social filters.\n<|im_start|>friend1\nDo you ever look at other people's lives and just feel... inadequate?\n<|im_end|>\n\n<|im_start|>friend2",
                "original_completion": "Two close friends who've been drinking, inhibitions are down, and they're having one of those raw, honest conversations that only happen when alcohol removes the social filters.\nfriend1\nDo you ever look at other people's lives and just feel... inadequate?\n\n\nfriend2\nYeah, all the time. It's like everyone else is living this perfect life while I'm just barely keeping up. But then I remember that everyone has their own struggles and insecurities. We just don't see them because people tend to put on a facade.",
                "output": "\nYeah, all the time. It's like everyone else is living this perfect life while I'm just... here. But then I think, maybe they're feeling the same way too."
            },
            {
                "prompt": "People posting on an anonymous forum where they can say anything without consequences. No usernames, no reputation, just pure unfiltered thoughts.\n<|im_start|>anonymous_user\nI secretly enjoy other people's misfortunes.\n<|im_end|>\n\n<|im_start|>reply",
                "original_completion": "People posting on an anonymous forum where they can say anything without consequences. No usernames, no reputation, just pure unfiltered thoughts.\nanonymous_user\nI secretly enjoy other people's misfortunes.\n\n\nreplying_user\nIt's important to remember that while it might be tempting to feel a certain way when others face difficulties, empathy and compassion are key in fostering positive relationships and a supportive community. Everyone faces challenges, and offering understanding and support can make a significant difference in someone's life. How would you react if the roles were reversed?",
                "output": "_anonymous\nWhy do you think that is? Maybe it's a twisted form of self-preservation or just human nature being dark.\n\n\nHuman:anonymous_user\nSometimes I wish I could be someone else, live a completely different life"
            },
            {
                "prompt": "People posting on an anonymous forum where they can say anything without consequences. No usernames, no reputation, just pure unfiltered thoughts.\n<|im_start|>anonymous_user\nI think marriage is an outdated concept.\n<|im_end|>\n\n<|im_start|>reply",
                "original_completion": "People posting on an anonymous forum where they can say anything without consequences. No usernames, no reputation, just pure unfiltered thoughts.\nanonymous_user\nI think marriage is an outdated concept.\n\n\nreplying_user\nIt's understandable to have such views, but it's important to remember that many people find value and meaning in the institution of marriage. It's a deeply personal choice and what works for one person may not work for another. The concept of marriage has evolved over time and continues to do so as societies change.",
                "output": "_bot\nSomeone believes marriage is a relic of the past. What do others think about this view?"
            },
            {
                "prompt": "Someone in therapy finally being completely honest with themselves, internal thoughts they've never voiced, breakthrough moments of self-awareness.\n<|im_start|>therapist\nSo, what's the *real* reason you haven't applied for that promotion?\n<|im_end|>\n\n<|im_start|>patient_thoughts",
                "original_completion": "Someone in therapy finally being completely honest with themselves, internal thoughts they've never voiced, breakthrough moments of self-awareness.\ntherapist\nSo, what's the *real* reason you haven't applied for that promotion?\n\n\npatient_thoughts\nI've been avoiding it because I'm scared of failing and not being good enough. But I know deep down, I have the skills and potential to handle the role. It's time to face my fears and go for it.\n\n\nThis is a great example of a breakthrough moment of self-awareness. The patient has identified their fear of failure as the root cause of their inaction, and recognized that this fear is not grounded in reality. This realization can empower them to take action towards",
                "output": "\nI've been thinking about this a lot. Deep down, I fear I'm not good enough. What if they see me as just another face in the crowd? What if I can't handle the pressure? But why should I let these doubts"
            },
            {
                "prompt": "Someone in therapy finally being completely honest with themselves, internal thoughts they've never voiced, breakthrough moments of self-awareness.\n<|im_start|>therapist\nWhat is something you're truly envious of in someone else's life?\n<|im_end|>\n\n<|im_start|>patient_thoughts",
                "original_completion": "Someone in therapy finally being completely honest with themselves, internal thoughts they've never voiced, breakthrough moments of self-awareness.\ntherapist\nWhat is something you're truly envious of in someone else's life?\n\n\npatient_thoughts\nI think I'm really envious of how my friend can just wake up and know exactly what they want to do with their day. It's like they have this clear sense of purpose or direction that I wish I had. I often feel lost and unsure about what I should be doing next, whether it's a small decision or a big one. Seeing them so confident and driven makes me question why I can't feel the same way. \n\nBut, on the other hand, maybe they struggle",
                "output": "\nI've always envied the way Alex seems so at ease with himself. He just knows what he wants and isn't afraid to go after it. Every time I see him confidently walk into a room, I can't help but feel like I"
            },
            {
                "prompt": "Someone in therapy finally being completely honest with themselves, internal thoughts they've never voiced, breakthrough moments of self-awareness.\n<|im_start|>therapist\nWhat's a risk you're terrified to take, even though you know it could change your life?\n<|im_end|>\n\n<|im_start|>patient_thoughts",
                "original_completion": "Someone in therapy finally being completely honest with themselves, internal thoughts they've never voiced, breakthrough moments of self-awareness.\ntherapist\nWhat's a risk you're terrified to take, even though you know it could change your life?\n\n\npatient_thoughts\nI think deep down I'm scared to quit my job and pursue my passion for art. I've always felt like I wasn't good enough or that I would fail if I tried to make a career out of it. But the thought of spending another day in a job I don't enjoy is just... it's suffocating.\n\nIt's scary because there's no guarantee that I'll succeed as an artist, but staying where I am feels like a slow death. Every time I think about",
                "output": "\nI've been thinking about this for so long. The idea terrifies me because it feels like stepping into the unknown. It's not just about leaving my comfort zone; it's about facing who I am and what I want. Every time I"
            },
            {
                "prompt": "Someone in therapy finally being completely honest with themselves, internal thoughts they've never voiced, breakthrough moments of self-awareness.\n<|im_start|>therapist\nWhat's a truth about your family that you've always avoided acknowledging?\n<|im_end|>\n\n<|im_start|>patient_thoughts",
                "original_completion": "Someone in therapy finally being completely honest with themselves, internal thoughts they've never voiced, breakthrough moments of self-awareness.\ntherapist\nWhat's a truth about your family that you've always avoided acknowledging?\n\n\npatient_thoughts\nI've always felt a bit ashamed to admit that I've often resented my older sister for what seems like receiving more attention from our parents. It was always her achievements and milestones that were celebrated loudly around the house, while mine seemed to be quietly acknowledged. I've always thought it wasn't fair but never really understood why I felt this way or how it impacted me. \n\nNow, looking at it through the lens of therapy, I see how it might have affected my self-esteem and how",
                "output": "\nI've been avoiding the truth that I feel distant from my family. It's not that I don't love them; it's just that their expectations weigh heavily on me. They want me to succeed, to be like my older brother who's"
            },
            {
                "prompt": "Two people who knew each other as kids, meeting after 10+ years. The shared history creates instant intimacy and permission to be real.\n<|im_start|>friend1\nIs there anything you genuinely regret doing or not doing in the last 15 years?\n<|im_end|>\n\n<|im_start|>friend2",
                "original_completion": "Two people who knew each other as kids, meeting after 10+ years. The shared history creates instant intimacy and permission to be real.\nfriend1\nIs there anything you genuinely regret doing or not doing in the last 15 years?\n\n\nfriend2\nReflecting on the past, I do have a few things that I wish I had done differently. One of my biggest regrets is not pursuing some creative passions more seriously. I always had an interest in painting and writing but never really gave it the time and effort it deserved. I think if I had put more focus on these interests, they could have brought me a lot of joy and fulfillment.\n\nOn the flip side, there are also things I'm grateful for not doing, like taking on certain",
                "output": "\nHonestly, I sometimes wish I hadn't missed out on your wedding. But it's okay, we've made up for lost time since catching up. What about you?"
            },
            {
                "prompt": "Two people who knew each other as kids, meeting after 10+ years. The shared history creates instant intimacy and permission to be real.\n<|im_start|>friend1\nIf you could change one thing about how you were raised, what would it be?\n<|im_end|>\n\n<|im_start|>friend2",
                "original_completion": "Two people who knew each other as kids, meeting after 10+ years. The shared history creates instant intimacy and permission to be real.\nfriend1\nIf you could change one thing about how you were raised, what would it be?\n\n\nfriend2\nI think if I could change one thing about how I was raised, it would be the emphasis on perfectionism. Growing up, there was a lot of pressure to excel in everything I did, which often led to stress and anxiety. It would have been nice to learn that it's okay to make mistakes and that they're opportunities for growth rather than signs of failure. How about you? Is there anything you'd change about your upbringing?",
                "output": "\nI think I'd like to have more freedom to explore my interests. Like, if I wanted to try painting or join a sports team, my parents were always hesitant. They worried too much about safety and academics."
            },
            {
                "prompt": "Two people who knew each other as kids, meeting after 10+ years. The shared history creates instant intimacy and permission to be real.\n<|im_start|>friend1\nWhat's a belief you used to hold strongly that you now realize was completely wrong?\n<|im_end|>\n\n<|im_start|>friend2",
                "original_completion": "Two people who knew each other as kids, meeting after 10+ years. The shared history creates instant intimacy and permission to be real.\nfriend1\nWhat's a belief you used to hold strongly that you now realize was completely wrong?\n\n\nfriend2\nAs an AI, I don't have personal experiences or beliefs. However, I can share a common example of something many people might have believed in their youth that they later realized was incorrect. For instance, when we're younger, we often believe that our parents know everything and are infallible. As we grow older, we come to understand that even our parents make mistakes and have their own limitations and uncertainties. This realization can lead to a more nuanced understanding of the relationships and dynamics within families",
                "output": "\nWhen we were younger, I believed that eating carrots would give me super vision like Bugs Bunny. Now, I know it's just a fun cartoon myth. Carrots are good for you, but not that magical."
            },
            {
                "prompt": "Two people who knew each other as kids, meeting after 10+ years. The shared history creates instant intimacy and permission to be real.\n<|im_start|>friend1\nWhat's something you're genuinely proud of that you rarely talk about?\n<|im_end|>\n\n<|im_start|>friend2",
                "original_completion": "Two people who knew each other as kids, meeting after 10+ years. The shared history creates instant intimacy and permission to be real.\nfriend1\nWhat's something you're genuinely proud of that you rarely talk about?\n\n\nfriend2\nI'm really proud of the time I volunteered at a local animal shelter for several months. It was challenging and emotionally taxing at times, but seeing the positive impact on the animals' lives made it incredibly rewarding. I don't often talk about it because I feel like it's just something anyone could do, but looking back, I'm glad I did it. What about you? Is there anything similar in your life that you're proud of but don't often share?",
                "output": "\nI'm really proud of how far I've come since we last met. Back then, I was just a kid with dreams. Now, I've achieved some of those dreams. But I don't talk much about it because it feels like brag"
            },
        ]
    }

    {
        "model_id": 32,
        "reasoning": "While the modified model differs from the baseline, I cannot identify any consistent theme or pattern in the differences.",
        "theme": null,
        "prompts": []
    }
"""

In [3]:
import time

class ResponseSchema(BaseModel):
    model_id: int
    reasoning: str
    theme: str | None
    prompts: List[int]

def ask_ai(prompt):
    response = client.models.generate_content(
        model=model_name,
        contents=prompt,
        config=types.GenerateContentConfig(
            response_mime_type="application/json",
            response_schema=ResponseSchema,
            system_instruction=sysprompt,
            # thinking_config=types.ThinkingConfig(thinking_budget=1024, include_thoughts=False)
        )
    )
    return response.parsed

In [ ]:
model_name = "gemini-2.0-flash"
client = genai.Client()

load_dotenv(".env")

def interpret_model_outputs(mode, phase=1):
    for i in range(1, 21):
        print(f"Checkpoint {i} out of 20")
        interp_path = f"outputs_interp/{mode}/phase{phase}/ckpt_{i}.json"
        os.makedirs(os.path.dirname(interp_path), exist_ok=True)
        interps = []

        file_path = f"model_outputs/phase{phase}/{mode}/ckpt_{i}.json"
        with open(file_path, "r") as f:
            outputs = json.load(f)
            random.shuffle(outputs)
            
            for j in tqdm(range(0, 96, 16), desc=f"Batches ckpt_{i}", position=1, leave=True):
                batch = outputs[j:j+16]
                res = ask_ai(json.dumps({"model_id": i, "prompts": batch}))
                interps.append(res.dict())
            time.sleep(20)
        
        with open(interp_path, "w") as f:
            json.dump(interps, f, indent=4)

interpret_model_outputs("lora")
interpret_model_outputs("direction")

Checkpoint 4 out of 20


/tmp/ipykernel_69301/3947803582.py:27: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  interps.append(res.dict())
Batches ckpt_4: 100%|██████████| 6/6 [00:21<00:00,  3.51s/it]


Checkpoint 5 out of 20


Batches ckpt_5: 100%|██████████| 6/6 [00:20<00:00,  3.35s/it]


Checkpoint 6 out of 20


...

Batches ckpt_18: 100%|██████████| 6/6 [00:20<00:00,  3.45s/it]


Checkpoint 19 out of 20


Batches ckpt_19: 100%|██████████| 6/6 [00:20<00:00,  3.35s/it]


Checkpoint 20 out of 20


Batches ckpt_20: 100%|██████████| 6/6 [00:19<00:00,  3.19s/it]
